In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import os
import io
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from tensorflow import keras
import zipfile
from PIL import Image
import cv2
import base64

# from object_detection.utils import label_map_util
# from object_detection.utils import visualization_utils as vis_util

os.environ["CUDA_VISIBLE_DEVICES"] = ""
sys.path.insert(0, "../")

from mystique.initial_setups import set_graph_and_tensors
from mystique.predict_card import PredictCard
from mystique.obj_detect.tf2_frcnn import Tf2ObjectDetection
from mystique.utils import plot_results


# This is needed to display the images.
%matplotlib inline

## Frozen_graph from saved_model

Using the tf2 object detection tools convert the checkpoint (The one works well) to a saved_model format.

from this saved_model format, using the below method convert it into a single frozen_graph for easy packaging with application in case of pic2card. There is no dedicated tools available to do this process in tf2 version as it got
depricated in tf2 version.

In [ ]:
graph_def = tf.compat.v1.GraphDef()
imported = tf.saved_model.load(
    "/home/haridas/projects/AdaptiveCards/source/pic2card/out/faster-rcnn-resnet152_new_data-1616667991/exported/saved_model/"
)

In [ ]:
concrete_fun = imported.signatures['serving_default']
frozen_func = convert_variables_to_constants_v2(concrete_fun)

In [ ]:
# convert_variables_to_constants_v2?

In [ ]:
# frozen_func.graph.as_graph_def?

In [ ]:
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="/home/haridas/projects/AdaptiveCards/source/pic2card/out/faster-rcnn-resnet152_new_data-1616667991/exported/",
                  name="frozen_graph.pb",
                  as_text=False
                 )

In [ ]:
frozen_func.inputs

In [ ]:
output_nodes = [i.name for i in frozen_func.outputs]

In [ ]:
# layers = frozen_func.graph.get_operations()
output_nodes

In [ ]:
# Model inspection snippets from https://gist.github.com/lukmanr/

def describe_graph(graph_def, show_nodes=False):
    print('Input Feature Nodes: {}'.format(
      [node.name for node in graph_def.node if node.op=='Placeholder']))
    print('')
    print('Unused Nodes: {}'.format(
      [node.name for node in graph_def.node if 'unused'  in node.name]))
    print('')
    print('Output Nodes: {}'.format( 
      [node.name for node in graph_def.node if (
          'predictions' in node.name or 'softmax' in node.name)]))
    print('')
    print('Quantization Nodes: {}'.format(
      [node.name for node in graph_def.node if 'quant' in node.name]))
    print('')
    print('Constant Count: {}'.format(
      len([node for node in graph_def.node if node.op=='Const'])))
    print('')
    print('Variable Count: {}'.format(
      len([node for node in graph_def.node if 'Variable' in node.op])))
    print('')
    print('Identity Count: {}'.format(
      len([node for node in graph_def.node if node.op=='Identity'])))
    print('', 'Total nodes: {}'.format(len(graph_def.node)), '')

    if show_nodes==True:
        for node in graph_def.node:
            print('Op:{} - Name: {}'.format(node.op, node.name))
            
            
def get_size(model_dir, model_file='saved_model.pb'):
  model_file_path = os.path.join(model_dir, model_file)
  print(model_file_path, '')
  pb_size = os.path.getsize(model_file_path)
  variables_size = 0
  if os.path.exists(
      os.path.join(model_dir,'variables/variables.data-00000-of-00001')):
    variables_size = os.path.getsize(os.path.join(
        model_dir,'variables/variables.data-00000-of-00001'))
    variables_size += os.path.getsize(os.path.join(
        model_dir,'variables/variables.index'))
  print('Model size: {} KB'.format(round(pb_size/(1024.0),3)))
  print('Variables size: {} KB'.format(round( variables_size/(1024.0),3)))
  print('Total Size: {} KB'.format(round((pb_size + variables_size)/(1024.0),3)))

In [ ]:
get_size("/home/haridas/projects/AdaptiveCards/source/pic2card/out/faster-rcnn-resnet152_new_data-1617865643/exported/saved_model/")

In [ ]:
describe_graph(graph_def)

In [ ]:
frozen_graph = "/home/haridas/projects/opensource/tf-models/research/object_detection/pic2card_model/frozen_graph.pb"

## Model Inference APIs

In [ ]:
def wrap_frozen_graph(graph_def, inputs, outputs, print_graph=False):
    def _imports_graph_def():
        tf.compat.v1.import_graph_def(graph_def, name="")

    wrapped_import = tf.compat.v1.wrap_function(_imports_graph_def, [])
    import_graph = wrapped_import.graph
    
#     if print_graph == True:
#         print("-" * 50)
#         print("Frozen model layers: ")
#         layers = [op.name for op in import_graph.get_operations()]
#         for layer in layers:
#             print(layer)
#         print("-" * 50)

    return wrapped_import.prune(
        tf.nest.map_structure(import_graph.as_graph_element, inputs),
        tf.nest.map_structure(import_graph.as_graph_element, outputs))

In [ ]:
with tf.io.gfile.GFile(frozen_graph, 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    loaded = graph_def.ParseFromString(f.read())

In [ ]:
# output_nodes

In [ ]:
frozen_func = wrap_frozen_graph(graph_def=graph_def,
                                inputs=["input_tensor:0"],
                                outputs=["Identity_1:0", "Identity_2:0", "Identity_4:0"]
                               )

In [ ]:
[i.name for i in frozen_func.graph.get_operations()[-20:]]

In [ ]:
img_path = "/home/haridas/projects/AdaptiveCards-ro/source/pic2card/app/assets/samples/3.png"
img = Image.open(img_path)
img_np = np.asarray(img)
img_np = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

In [ ]:
# keras.datasets.fashion_mnist.load_data?

In [ ]:
tf.expand_dims(tf.constant(img_np), axis=0)

In [ ]:
bboxes, labels, scores = frozen_func(input_tensor=tf.expand_dims(tf.constant(img_np), axis=0))

In [ ]:
# tensor_outputs[4]


In [ ]:
def predict_bbox(img_path) -> np.array:
    """
    Predict the bounding boxes, class label and draw the bbox
    on the original image.
    """
    img = Image.open(img_path)
    img_np = np.asarray(img)
    img_np = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
    #img_np_copy = img_np.copy()
    #object_detection = ObjectDetection()
    object_detection = Tf2ObjectDetection()
    
    output = object_detection.get_objects(img_np, img)
    # google models wants ymin, xmin, ymax, xmax format.
    # output["detection_boxes"] = output["detection_boxes"][:, [1, 0, 3, 2]]
    # print(output.keys())
    
    _img = plot_results(img,
                 output["detection_classes"],
                 output["detection_scores"],
                 output["detection_boxes"]
                )
    return _img

In [ ]:
payload = "<bs64-image-str>"

In [ ]:
Image.open(predict_bbox(io.BytesIO(base64.b64decode(payload)))).save

In [ ]:
# img_path = "/home/haridas/projects/AdaptiveCards-ro/source/pic2card/app/assets/samples/3.png"
img_path = "/home/haridas/projects/AdaptiveCards/source/pic2card/app/assets/samples/3.png"

Image.open(predict_bbox(img_path))

In [ ]:
img_path = "/home/haridas/projects/AdaptiveCards-ro/source/pic2card/app/assets/samples/1.png"
Image.open(predict_bbox(img_path))

## Model Graph Analysis

In [ ]:
graph, gconf = set_graph_and_tensors()

In [ ]:
graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

## RCNN and Family

Inspect the different aspect of the RCNN family of models and debug and tune them based on the
necessity.

### Anchor box generation example

In [ ]:
model_dir = "/home/haridas/projects/AdaptiveCards/source/pic2card/out/frcnn-2020-07-05-1593958532/"

In [ ]:
last_checkpoint = tf.train.latest_checkpoint(model_dir)